# DSS2ETCBC: the lexemes and parts of speech of the biblical scrolls

By Martijn Naaijer and Jarod Jacobs

One of the goals of the CACCHT project is to integrate the scrolls in the ETCBC database by encoding them in the ETCBC format. In a previous blogpost we explained how POS tagging of Hebrew texts can be done using an LSTM network. In this blogpost we show how the lexemes of the words in the biblical scrolls can be converted from the encoding done by Abegg to the ETCBC lexemes.

## General approach

The text-fabric package containing the Dead Sea Scrolls has a variety of word features, such as number, gender, person, and lexeme. Some of these can be converted straightforwardly to ETCBC format. An example is number. We assume that verb forms in the first person in the dss package correspond one to one with the first person in the ETCBC database. 

There are features, for which this correspondence cannot be used straightforwardly. An example is the word feature part of speech. In the dss package there is the value "ptcl" (particle). This value corresponds with "conj" (conjunction), and "prep" (preposition), among other things.

Something similar is going on with the feature lexeme. In the ETCBC database, the lexemes are based on KBL lexicon, but in the course of time a whole range of improvements have been implemented, based on ongoing research. Therefore, it is understandable, that there is not always a one to one correspondence between the lexemes of the dss package and the BHSA.

So, how can we assign the ETCBC values of the feature lexeme to the scrolls? In the case of the biblical scrolls one can simply use lexemes used for the same words in corresponding verses. For instance, the scroll 4Q2 contains the text of Genesis. Its text of verse 1:1 is BR>CJT BR> >LHJM >T HCMJM W>T H>RY, which is identical to the text of Genesis 1:1 in the BHSA. In this case, we can simply give each word the lexeme of the same word in the BHSA. In many cases, however, the text of the scrolls deviates more or less with that of the BHSA.

This problem is solved by using sequence alignment. This means that two strings are arranged in such a way that similar parts are identified. Alignment of sequences is used often in biology, if one wants to identify similarities and differences in DNA strings or proteins. Sequence alignment techniques are often based on dynamic programming, such as the Smith Waterman Algorithm. In the package [biopython](https://towardsdatascience.com/pairwise-sequence-alignment-using-biopython-d1a9d0ba861f) a whole range of algorithms for the study of biological sequences are implemented. We use this package, and the module pairwise2, which is contained in it, to align biblical verses in the DSS and the BHSA.

In practice , this looks as follows. As an example we look at Isaiah 48:2 in the BHSA and 1QIsaa. 

    KJ- M <JR H Q-DC NQR>W W <L >L-HJ JFR>L NSMKW JHWH YB>WT CMW   BHSA
    
    KJ> M <JR H QWDC NQR>W W <L >LWHJ JFR>L NSMKW JHWH YB>WT CMW   1QIsaa


It is clear that the text of the BHSA and 1QIsaa is very similar, but there are also some differences. Similar parts in the verses are put together, resulting in two sequences of equal length. On the place of the extra matres lectiones in 1QIsaa, one finds a gap ("-") in the BHSA.

Now, we look at every character in 1QIsaa, and check the lexeme of the corresponding character in the BHSA. If more than half of the characters in a word in the scroll corresponds with one lexeme in the BHSA, we give the value of this lexeme to the word in the scroll. In the case of the first word in Isaiah 48:2, the K and J correspond to a character in the corresponding word with the lexeme KJ, and the character > does not correspond with a word in the BHSA. The result is that 2 out of 3 characters correspond with the lexeme KJ, which is more than 50%, so the first word KJ> in this verse of the scroll gets the value KJ. This approach works well in practice, but it does not result in a value for each word. 

An example is Isaiah 42:23 in 1QIsaa: 

    MJ- BKM- --J>ZJN Z->T --JQCB W JCM< L >XWR    BHSA

    MJ> BKMH W J>ZJN ZW>T W JQCB W JCM< L >XWR    1QIsaa

In this case, the word "W" occurs three times in 1QIsaa, and only once in the BHSA. This means, that in the first two cases we cannot give these words an ETCBC lexeme. We can proceed by checking the lexeme "W" has in these cases in the dss module, which is "W:", and then find out with which lexeme "W:" corresponds in the BHSA most often. This is the BHSA lexeme "W", which can be given to the unmatched words "W".

This procedure works well in practice, but it is not infallible. In some cases the alignment is unfortunate, in other cases there is no one to one mapping between lexemes or the dss package has different definitions of what is a word. For instance, in the case of place names consisting of two words, the dss package treats them as distinct words, whereas the BHSA has one lexeme in general. To bring the resulting dataset to "ETCBC standard", more steps are needed. These can consist of additional automatic processing and/or manual cleaning.

The same approach is used for the feature part of speech. You can find the resulting dataset in the [github repository](https://github.com/ETCBC/DSS2ETCBC).

In [1]:
import collections
from pprint import pprint
import pandas as pd

The package biopython is used for aligning sequences

In [2]:
from Bio import pairwise2
from Bio.Seq import Seq

Now the dss package is loaded. The classes L, T and F, that are used here are renamed. In that case we can use them together

In [3]:
from tf.app import use
A = use('dss', hoist=globals())

# give the relevant classes for the DSS new names
Ldss = L
Tdss = T
Fdss = F

	connecting to online GitHub repo annotation/app-dss ... no internet
The offline TF-app may not be the latest
Using TF-app in C:\Users\geitb/text-fabric-data/annotation/app-dss/code:
	rv0.6=#304d66fd7eab50bbe4de8505c24d8b3eca30b1f1 (latest? release)
	connecting to online GitHub repo etcbc/dss ... no internet
The offline data may not be the latest
Using data in C:\Users\geitb/text-fabric-data/etcbc/dss/tf/0.6:
	rv0.6=#9b52e40a8a36391b60807357fa94343c510bdee0 (latest? release)
	connecting to online GitHub repo etcbc/dss ... no internet
The offline data may not be the latest
Using data in C:\Users\geitb/text-fabric-data/etcbc/dss/parallels/tf/0.6:
	rv0.6=#9b52e40a8a36391b60807357fa94343c510bdee0 (latest? release)
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used


In [4]:
from tf.app import use
A = use('bhsa', hoist=globals())

	connecting to online GitHub repo annotation/app-bhsa ... no internet
The offline TF-app may not be the latest
Using TF-app in C:\Users\geitb/text-fabric-data/annotation/app-bhsa/code:
	rv1.2=#5fdf1778d51d938bfe80b37b415e36618e50190c (latest? release)
	connecting to online GitHub repo etcbc/bhsa ... no internet
The offline data may not be the latest
Using data in C:\Users\geitb/text-fabric-data/etcbc/bhsa/tf/c:
	rv1.6=#bac4a9f5a2bbdede96ba6caea45e762fe88f88c5 (latest? release)
	connecting to online GitHub repo etcbc/phono ... no internet
The offline data may not be the latest
Using data in C:\Users\geitb/text-fabric-data/etcbc/phono/tf/c:
	r1.2 (latest? release)
	connecting to online GitHub repo etcbc/parallels ... no internet
The offline data may not be the latest
Using data in C:\Users\geitb/text-fabric-data/etcbc/parallels/tf/c:
	r1.2 (latest? release)
   |     0.01s No structure info in otext, the structure part of the T-API cannot be used


The dictionary book_dict_bhsa_dss provides a mapping between the booknames of the BHSA and dss packages. 

In [5]:
book_dict_bhsa_dss = {'Genesis':      'Gen',
             'Exodus':       'Ex',
             'Leviticus':    'Lev',
             'Numbers':      'Num',
             'Deuteronomy':  'Deut',
             'Joshua':       'Josh',
             'Judges':       'Judg',
             '1_Samuel':     '1Sam',
             '2_Samuel':     '2Sam',
             '1_Kings':      '1Kgs',
             '2_Kings':      '2Kgs',
             'Isaiah':       'Is',
             'Jeremiah':     'Jer',
             'Ezekiel':      'Ezek',
             'Hosea':        'Hos',
             'Joel':         'Joel',
             'Amos':         'Amos',
             'Obadiah':      'Obad',
             'Jonah':        'Jonah',
             'Micah':        'Mic',
             'Nahum':        'Nah',
             'Habakkuk':     'Hab',
             'Zephaniah':    'Zeph',
             'Haggai':       'Hag',
             'Zechariah':    'Zech',
             'Malachi':      'Mal',
             'Psalms':       'Ps',
             'Job':          'Job',
             'Proverbs':     'Prov',
             'Ruth':         'Ruth',
             'Song_of_songs':'Song',
             'Ecclesiastes': 'Eccl',
             'Lamentations': 'Lam',
             'Daniel':       'Dan',
             'Ezra':         'Ezra',
             '2_Chronicles': '2Chr'
            }

And the reversed way can be useful as well.

In [6]:
book_dict_dss_bhsa = {v: k for k, v in book_dict_bhsa_dss.items()}
print(book_dict_dss_bhsa)

{'Gen': 'Genesis', 'Ex': 'Exodus', 'Lev': 'Leviticus', 'Num': 'Numbers', 'Deut': 'Deuteronomy', 'Josh': 'Joshua', 'Judg': 'Judges', '1Sam': '1_Samuel', '2Sam': '2_Samuel', '1Kgs': '1_Kings', '2Kgs': '2_Kings', 'Is': 'Isaiah', 'Jer': 'Jeremiah', 'Ezek': 'Ezekiel', 'Hos': 'Hosea', 'Joel': 'Joel', 'Amos': 'Amos', 'Obad': 'Obadiah', 'Jonah': 'Jonah', 'Mic': 'Micah', 'Nah': 'Nahum', 'Hab': 'Habakkuk', 'Zeph': 'Zephaniah', 'Hag': 'Haggai', 'Zech': 'Zechariah', 'Mal': 'Malachi', 'Ps': 'Psalms', 'Job': 'Job', 'Prov': 'Proverbs', 'Ruth': 'Ruth', 'Song': 'Song_of_songs', 'Eccl': 'Ecclesiastes', 'Lam': 'Lamentations', 'Dan': 'Daniel', 'Ezra': 'Ezra', '2Chr': '2_Chronicles'}


We define some helper functions. align_verses takes two sequences das input and aligns them. It returns the aligned sequences.

In [7]:
def align_verses(bhsa_data, dss_data):
    
    seq_bhsa = ' '.join(bhsa_data).strip()
    seq_dss = ' '.join(dss_data).strip()
        
    seq1 = Seq(seq_bhsa) 
    seq2 = Seq(seq_dss)
    
    alignments = pairwise2.align.globalxx(seq1, seq2)
    
    bhsa_al = (alignments[0][0]).strip(' ')
    dss_al = (alignments[0][1]).strip(' ')
        
    return bhsa_al, dss_al

The function most_frequent takes a list with lexemes as input, and returns the most frequent lexeme, together with its frequency in the list.

In [8]:
from collections import Counter 
  
def most_frequent(lex_list): 
    occurrence_count = Counter(lex_list) 
    lex = occurrence_count.most_common(1)[0][0]
    count = occurrence_count.most_common(1)[0][1]
    return(lex, count)

In the function produce_value, the value of the POS or lexeme in the ETCBC format is retrieved.

In [9]:
def produce_value(key, chars_to_feat_etcbc, chars_to_feat_dss):
    # check for each consonant what the value of the feature is of the word of the corresponding consonant in the BHSA 
    if len(chars_to_feat_etcbc[(key[0], key[1], key[2])]) > 0:
        all_feat_etcbc = chars_to_feat_etcbc[(key[0], key[1], key[2])]
        
        # check for a word to which lexeme in the bhsa it corresponds with most of its characters
        feat_etcbc_proposed, count = most_frequent(all_feat_etcbc)
        
        # an etcbc-lexeme is assigned only to a word if more than half of its consonants
        # corresponds with a word in the BHSA    
        if len(list(word)) == 0:
            feat_etcbc = ''

        elif (count / len(list(word))) > 0.5:
            feat_etcbc = feat_etcbc_proposed

        else:
            feat_etcbc = ''
            
    else:
        feat_etcbc = ''
        
    if len(chars_to_feat_dss[key]) > 0:
        
        all_feat_dss = chars_to_feat_dss[key]
        feat_dss, count = most_frequent(all_feat_dss)
        
    else:
        feat_dss = ''
        
    return feat_dss, feat_etcbc

## Prepare scrolls


In the next cell the text and lexemes of the Great Isaiah scroll are extracted from the DSS package, and some minor manipulations are done with the text.

For every character in the text of the scroll 1QIsaa, we look up what the lexeme is of the word in which the character occurs. This information is saved in the dictionary lexemes_dss. The keys of this dict are the verse numbers of Isaiah. Each value is a list with the lexemes, one for each character in the verse.

Note that the data retrieved in the following cell consists partly of reconstructions of scrolls. Other features in the package, not discussed here, deal with which part of the text can be read on the scrolls and which part is reconstructed.

In [19]:
# In dss_data_dict, the text of each verse in the biblical scrolls is collected
dss_data_dict = collections.defaultdict(lambda: collections.defaultdict(list))
lexemes_dss = collections.defaultdict(lambda: collections.defaultdict(list))
pos_dss = collections.defaultdict(lambda: collections.defaultdict(list))
ids_dss = collections.defaultdict(lambda: collections.defaultdict(list))

for scr in Fdss.otype.s('scroll'):
    scroll_name = Tdss.scrollName(scr)
    
    words = Ldss.d(scr, 'word')
        
    for w in words:
            
        # exclude fragmentary data, these chapters start with 'f'
        chapter = Ldss.u(w, 'chapter')
        
        bo = Fdss.book.v(w) 
        
        if bo == None or bo not in book_dict_dss_bhsa:
            continue
        
        if (Fdss.chapter.v(w))[0] == 'f':
            continue
        
        # Do a bit of preprocessing
        if Fdss.glyphe.v(w) != None:
            
            lexeme = Fdss.glexe.v(w)  
            glyphs = Fdss.glyphe.v(w)
            
            # dummy value, check what happens here
            if lexeme == None:
                print(glyphs, scroll_name, int(Fdss.chapter.v(w)), int(Fdss.verse.v(w)))
                lexeme = 'XXX'
                
            # the consonant '#' is used for both 'C' and 'F'. We check in the lexeme
            # to which of the two alternatives it should be converted. This appproach is crude, 
            # but works well in practice. There is only one word with both F and C in the lexeme: >RTX##T> >AR:T.AX:CAF:T.:> in 4Q117
            if '#' in glyphs:  
                # hardcode the single word with both 'C' and 'F' in the lexeme.
                if glyphs == '>RTX##T>':
                    glyphs = '>RTXCFT>'
                    
                elif 'F' in lexeme:
                    glyphs = glyphs.replace('#', 'F')
                    
                # cases in wich 'C' occurs in lexeme or morphology
                else:                        
                    glyphs = glyphs.replace('#', 'C')                        
            
            # Some characters are removed or replaced, in the case of 'k', 'n', 'm', 'y', 'p', it concerns final consonants of words.
            glyphs = glyphs.replace(u'\xa0', u' ').replace("'", "").replace("k", "K").replace("n", "N").replace("m", "M").replace("y", "Y").replace("p", "P")   
                
            dss_book = Fdss.book.v(w)
            bhsa_book_name = book_dict_dss_bhsa[dss_book]
            
            # replace(' ', '') is needed for strange case in Exodus 13:16 with a space in the word
            dss_data_dict[(bhsa_book_name, int(Fdss.chapter.v(w)), int(Fdss.verse.v(w)))][scroll_name].append(glyphs.replace(' ', ''))
            
            ids_dss[bhsa_book_name, int(Fdss.chapter.v(w)), int(Fdss.verse.v(w))][scroll_name].append(w)
            
            # retrieve POS and lexeme of every character of every word in the scrolls and save in a dictionary
            for character in glyphs:   
                pos_dss[bhsa_book_name, int(Fdss.chapter.v(w)), int(Fdss.verse.v(w))][scroll_name].append(Fdss.sp.v(w))
                lexemes_dss[bhsa_book_name, int(Fdss.chapter.v(w)), int(Fdss.verse.v(w))][scroll_name].append(Fdss.glexe.v(w))


'''''' 1Qisaa 19 17
'''''' 1Qisaa 19 17
'''' 1Qisaa 22 5
''''''''''' 1Qisaa 22 21
'''' 1Qisaa 24 14
'''' 1Qisaa 27 11
> 1Qisaa 34 17
B 1Qisaa 35 2
> 1Qisaa 37 6
B 1Qisaa 37 6
P 1Qisaa 44 12
'''' 1Qisaa 55 10
'HMH 1Qisaa 59 8
''''' 4Q1 1 11
''''' 4Q1 1 12
''''' 4Q1 1 14
MY 4Q11 3 18
>< 4Q11 3 20
J# 4Q11 14 16
MYR 4Q11 14 18
XQ 4Q11 18 20
M> 4Q11 18 21
B< 4Q12 26 23
>WMRW 4Q12 26 28
J# 4Q22 10 20
MY 4Q22 10 21
MY 4Q22 10 22
B 4Q22 10 24
''''''' 4Q35 1 11
'''''''' 4Q35 33 20
> 4Q38c 13 2
''''''' 4Q41 5 5
''''' 4Q41 5 5
''''''''' 4Q41 5 12
'''' 4Q41 5 14
''''''' 4Q41 5 15
''''''''''''''' 4Q41 5 15
'''''''''''' 4Q41 5 15
''''' 4Q41 20 11
'''''''''' 4Q41 5 16
'''' 4Q41 5 33
>B 4Q45 15 9
XQT 4Q45 28 15
MHW 4Q45 28 20
J 4Q46 26 14
m 4Q46 26 15
T 4Q46 26 15
''''''' 4Q56 19 9
''''''' 4Q56 19 10
> 4Q70 7 30
> 4Q70 7 30
B 4Q70 7 31
G 4Q70 7 31
D 4Q70 7 33
H 4Q70 8 1
W 4Q70 8 2
Z 4Q70 8 3
'''''' 4Q72 8 22
'''' 4Q72 8 22
''''' 4Q72 8 23
L> 4Q72 24 6
>TW# 4Q72 24 6
''' 4Q72 25 26
''' 4Q72 25 26
''' 4

## Prepare BHSA

The same is done for all the characters in the text of Isaiah in the BHSA. Also, a list with the verses of Isaiah in the BHSA is made. Finally, for each verse, the consonantal structure of each word is collected per verse in the dictionary bhsa_data_dict.

In [17]:
all_verses = []
lexemes_bhsa = collections.defaultdict(list)
pos_bhsa = collections.defaultdict(list)

bhsa_data_dict = collections.defaultdict(list)

for w in F.otype.s('word'):
    
    # Remove words without consonantal representation in the text (elided he). 
    if F.g_cons.v(w) == '':
        continue
    
    bo, ch, ve = T.sectionFromNode(w)
    cl = L.u(w, "clause")[0]
    words_in_cl = L.d(cl, "word")
        
    # use feature g_cons for consonantal representation of words
    bhsa_data_dict[(bo, ch, ve)].append(F.g_cons.v(w))
        
    # loop over consonants and get lexeme of each consonant in a word
    for cons in F.g_cons.v(w):
        lexemes_bhsa[(bo, ch, ve)].append(F.lex.v(w)) 
        pos_bhsa[(bo, ch, ve)].append(F.sp.v(w))
        
    if (bo, ch, ve) not in all_verses:
        all_verses.append((bo, ch, ve))
    

## Align verses

In the following cell, the verses are aligned and characters are compared. 

In [18]:
chars_to_lexemes_etcbc = collections.defaultdict(list)
chars_to_lexemes_dss = collections.defaultdict(list)

chars_to_pos_etcbc = collections.defaultdict(list)
chars_to_pos_dss = collections.defaultdict(list)
chars_per_word =  collections.defaultdict(list)

all_verses = list(set(all_verses))

# loop over verses in BHSA
for verse in all_verses:
    
    # check if verse occurs in the dss package
    if verse in dss_data_dict:
        
        scrolls = (dss_data_dict[verse]).keys()
        
        for scroll in scrolls:
            
            # get the text of the verse
            bhsa_data = bhsa_data_dict[verse]
            dss_data = dss_data_dict[verse][scroll]
            
            # all pairs of verses in the BHSA and scrolls are aligned
            bhsa_al, dss_al = align_verses(bhsa_data, dss_data)
             
            print(verse)
            print('BHSA')
            print(bhsa_al)
            print(scroll)
            print(dss_al)
            print(' ')
        
            # some indexes are initialized, these keep track of how many consonants have been observed 
            # in both aligned sequences
            ind_dss = 0
            ind_bhsa = 0
        
            dss_word_ind = 0
        
            # loop over all characters in the BHSA verse
            for pos in range(len(bhsa_al)):
            
                # for each character in the BHSA sequence, it is cheched what the character is in the DSS sequence.
                # Now, a number of scenarios are defined. Most scenarios are not so exciting, e.g.
                # if the character is a space in both sequences, then move further
                if bhsa_al[pos] == ' ' and dss_al[pos] == ' ':
                    
                    dss_word_ind += 1
            
                elif bhsa_al[pos] == '-' and dss_al[pos] == ' ':
                    
                    dss_word_ind += 1
                
                elif bhsa_al[pos] == ' ' and dss_al[pos] == '-':
                    
                    chars_per_word[(scroll, verse, dss_word_ind)].append(dss_al[pos])
                               
                else:
                    if bhsa_al[pos] == '-':
                        
                        chars_per_word[(scroll, verse, dss_word_ind)].append(dss_al[pos])
                        chars_to_lexemes_dss[(scroll, verse, dss_word_ind)].append(lexemes_dss[verse][scroll][ind_dss])
                        chars_to_pos_dss[(scroll, verse, dss_word_ind)].append(pos_dss[verse][scroll][ind_dss])
                        
                        ind_dss += 1
                    
                    elif dss_al[pos] == '-':
                        
                        chars_per_word[(scroll, verse, dss_word_ind)].append(dss_al[pos])
                        
                        ind_bhsa += 1
                    
                    # Now the real matching is done
                    # For a matching consonant, it is checked in the dicts to which lexeme it corresponds
                    else:                        
                        
                        chars_to_lexemes_etcbc[(scroll, verse, dss_word_ind)].append(lexemes_bhsa[verse][ind_bhsa])
                        chars_to_pos_etcbc[(scroll, verse, dss_word_ind)].append(pos_bhsa[verse][ind_bhsa])
                        
                        chars_to_lexemes_dss[(scroll, verse, dss_word_ind)].append(lexemes_dss[verse][scroll][ind_dss])
                        chars_to_pos_dss[(scroll, verse, dss_word_ind)].append(pos_dss[verse][scroll][ind_dss])
                        chars_per_word[(scroll, verse, dss_word_ind)].append(dss_al[pos])
                    
                        ind_dss += 1
                        ind_bhsa += 1               

('Ezekiel', 1, 12)
BHSA
W >JC >L <BR PNJW JLKW >L >CR JHJH CMH H RWX L LKT JLKW L> JSBW B LKTN
4Q74
W >JC >L <BR PNJW JLKW >L >CR JHJH CMH H RWX L LKT JLKW L> JSBW B LKTN
 
('Isaiah', 28, 15)
BHSA
KJ >MRTM KRTNW BRJT >T MWT W <M C>WL <FJNW XZH CJ-V CWVP KJ <-B-R L-> JBW>NW KJ FMNW KZB MXSNW- W B CQR NSTRNW
1Qisaa
KJ >MRTM KRTNW BRJT >T MWT W <M C>WL <FJNW XZH C-WV CWVP KJ -JBWR LW> JBW>NW KJ FMNW KZB MXSN-J W B CQR NSTRNW
 
('Isaiah', 28, 15)
BHSA
KJ >MRTM KRTNW BRJT >T MWT W <M C>WL <FJNW XZH CJV CWVP KJ <BR L> JBW>NW KJ FMNW KZB MXSNW W B CQR NSTRNW
1Q8
---------KRTNW BRJT >T MWT W <M C>WL <FJNW XZH CJV CWVP KJ <BR L> JB->NW KJ FMNW KZB MXSNW W B CQR NSTRNW
 
('Leviticus', 25, 51)
BHSA
>M <WD RBWT B CNJM L PJHN JCJB G>LTW M KSP MQNTW
4Q24
>M <WD RBWT B CNJM L PJHN JCJB G>LTW M KSP MQNTW
 
('Isaiah', 38, 14)
BHSA
K SWS <GW-R KN >YPYP >HGH K JWNH- DLW <JNJ L MRWM >D-NJ <-CQH LJ --<RBNJ
1Qisaa
K SWS <-WGR KN >YPYP >HGH K JWN-> DLW <JNJ L MRWM >DWNJ <WCQH LJ W <RBNJ
 
('Isaiah', 38, 14)


('2_Samuel', 4, 4)
BHSA
W L JHWNTN BN C>WL BN NKH RGLJM BN XMC CNJM H--JH- B B-> CM-<T C>WL W JHWNTN --------M JZR<>L W TF>HW >MNTW W TNS W JHJ B XPZH L NWS W JPL W JPSX W CMW MPJBCT
4Q51
W L JHWNTN BN C>WL BN NKH RGLJM BN XMC CNJM -W JHJ B BW> CMW<T C>WL W JHWNTN BN C>WL M JZR<>L W TF>HW >MNTW W T------------------N-------------S--------------
 
('Isaiah', 3, 21)
BHSA
--H VB<WT W NZMJ H >P
1Qisaa
W H VB<WT W NZMJ H >P
 
('Isaiah', 3, 21)
BHSA
H VB<WT W NZMJ H >P
4Q56
H VB<WT W NZMJ H >P
 
('Micah', 7, 1)
BHSA
>LLJ LJ KJ HJJTJ K >SPJ QJY K <LLT BYJR >JN >CKWL L >KWL BKWRH >WTH NPCJ
Mur88
>LLJ LJ KJ HJJTJ K >SPJ QJY K <LLT BYJR >JN >CKWL L >KWL BKWRH >WTH NPCJ
 
('Deuteronomy', 16, 2)
BHSA
W ZBXT PSX L JHWH >LHJK Y>N W BQR B MQWM >CR JBXR JHWH L CKN CMW CM
4Q30
------------------------Y>N W BQR B MQWM >CR JBXR JHWH L CKN CMW CM
 
('Exodus', 3, 19)
BHSA
W >NJ JD<TJ KJ L> JTN >TKM MLK MYRJM L HLK W L> B JD XZQH
4Q1
W >NJ JD<TJ KJ L> JTN >TKM MLK MYRJM L HLK W L> B JD XZQH
 
('Exodus', 3, 

('Exodus', 24, 1)
BHSA
W >L MCH >MR <LH >L JHWH >TH W >HRN NDB W >BJHW> ------W --------CB<JM M ZQNJ JFR>L W HCTXWJTM M RXQ
4Q22
W >L MCH >MR <LH >L JHWH >TH W >HRN NDB W >BJHW> >L<ZR W >JTMR W CB<JM M ZQNJ JFR>L W HCTXWJTM M RXQ
 
('Psalms', 82, 7)
BHSA
>KN K >DM TMWTWN W K >XD H FRJM TPLW
Mas1e
>KN K >DM TMWTWN W K >XD H FRJM TPLW
 
('Deuteronomy', 24, 10)
BHSA
KJ TCH B R<K MC>T M>WMH L> TB> >L BJTW L <BV <BVW
1Q5
-------------MC>T M>WMH L> TB> >L BJTW L <BV <BVW
 
('Deuteronomy', 11, 30)
BHSA
H L> HMH B <BR H JRDN >XRJ DRK MBW> H CMC B >RY H KN<NJ H JCB B <RBH MWL H GLGL >YL >LWNJ MRH
1Q4
H L> HMH B <BR H JRDN >XRJ DRK MBW> H CMC B >RY H KN<NJ H JCB B <RBH MWL H GLG-------L-------
 
('Deuteronomy', 11, 30)
BHSA
H L> HMH B <BR H JRDN >XRJ DRK MBW> H CMC B >RY H KN<NJ H JCB B <RBH MWL H GLGL >YL >LWNJ MRH
1Q5
H L> HMH B <BR H JRDN >XRJ DRK MBW> H CMC B >RY H KN<NJ H JCB B <RBH MWL H GLGL >YL >LWNJ MRH
 
('Deuteronomy', 11, 30)
BHSA
H L> HMH B <BR H JRDN >XRJ DRK MBW> H CMC B >RY H KN<

KJ JHWH >LHJK MBJ>K >L >RY VWBH ------->RY NXLJ MJM <JNT W THMT JY>JM B BQ<H W B HR
4Q33
KJ JHWH >LHJK MBJ>K >L >RY VWBH W RXBH >RY NXLJ MJM <JNT W THMT JY>JM B BQ<H W B HR
 
('Deuteronomy', 8, 7)
BHSA
KJ JHWH >L-HJK- MBJ>K- >L >RY VWBH ------->RY NXLJ MJM <JN-T W TH-M-T J-Y>JM B BQ<H W B HR
4Q37
KJ JHWH >LWHJKH MBJ>KH >L >RY VWBH W RXBH >RY NXLJ MJM <JNWT W THWMWT JWY>JM B BQ<H W B HR
 
('Deuteronomy', 8, 7)
BHSA
KJ JHWH >L-HJK MBJ>K >L >RY VWBH ------->RY NXLJ MJM <JN-T W TH-M-T JY>JM B BQ<H W B HR
4Q41
KJ JHWH >LWHJK MBJ>K >L >RY VWBH W RXBH >RY NXLJ MJM <JNWT W THWMWT JY>JM B BQ<H W B HR
 
('Deuteronomy', 8, 7)
BHSA
KJ JHWH >LHJK MBJ>K >L >RY VWBH >RY NXLJ MJM <JNT W THMT JY>JM B BQ<H W B HR
5Q1
KJ JHWH >LHJK MBJ>K >L >RY VWBH >RY NXLJ MJM <JNT W THMT JY>JM B BQ<H W B HR
 
('Numbers', 32, 30)
BHSA
W >M L> J<BRW XLWYJM >TKM W N>XZW B TKKM B >RY KN<N
4Q23
W >M L> J<BRW XLWYJM >TKM W N>XZW B TKKM B >RY KN<N
 
('Numbers', 32, 30)
BHSA
W >M L-> J<B-RW XLWYJM >TKM- W- N>--XZ-----------W-

('Deuteronomy', 29, 17)
BHSA
PN JC BKM >JC >W >CH >W MCPXH >W CBV >CR LBBW PNH H JWM M <M JHWH >LHJNW L LKT L <BD >T >LHJ H GWJM H HM PN JC BKM CRC PRH R>C W L<NH
4Q30
------------------------MCPXH >W CBV >CR LBBW PNH H JWM M <M JHWH >LHJNW L LKT L <BD >T >LHJ H GWJM H HM PN JC BKM CRC PRH R>C W L<NH
 
('Obadiah', 1, 19)
BHSA
W JRCW H NGB >T HR <FW W H CPLH >T PLCTJM W JRCW >T FDH >PRJM W >T FDH CMRWN W BNJMN >T H GL<D
Mur88
W JRCW H NGB >T HR <FW W H CPLH >T PLCTJM W JRCW >T FDH >PRJM W >T FDH CMRWN W BNJMN >T H GL<D
 
('Psalms', 147, 20)
BHSA
L> <FH KN L KL GWJ W MCPVJM BL JD<WM HLLW JH
4Q86
L> <FH KN L KL GWJ W MCPVJM BL JD<WM HLLW JH
 
('Psalms', 147, 20)
BHSA
L-> <FH KN L K-L GWJ W MCPVJM BL J--D-<WM HLLW JH
11Q5
LW> <FH KN L KWL GWJ-- MCPVJM BL -HWDJ<-M HLLW JH
 
('Leviticus', 23, 18)
BHSA
W HQRBTM <L H LXM CB<T KBFJM TMJMM BNJ CNH W PR BN BQR >XD W >JLM CNJM JHJW <LH L JHWH W MNXTM W NSKJHM >CH RJX NJX-X L JHWH
4Q24
W HQRBTM <L H LXM CB<T KBFJM TMJMM BNJ CNH W PR BN BQR >XD W >J

...

('Isaiah', 14, 2)
BHSA
W LQXWM <MJM W HBJ>WM >L MQWMM W HTNXLWM BJT JFR>L <L >DMT JHWH L <BDJM W L CPXWT W HJW CBJM L CBJHM W RDW B NGFJHM
4Q59
W LQXWM <MJM W HBJ>WM >L MQWMM W HTNXLWM BJT JFR>L <L >DMT JHWH L <BDJM W L CPXWT W HJW CBJM L CBJHM W RDW B NGFJHM
 
('Leviticus', 11, 38)
BHSA
W KJ JTN MJM <L ZR< W NPL M NBLTM <LJW VM> HW> LKM
Mas1b
W KJ JTN MJM <L ZR< W NPL M NBLTM <LJW VM> HW> LKM
 
('1_Samuel', 1, 13)
BHSA
W XNH HJ> MDBRT <L LBH RQ FPTJH N<WT W QWLH L> JCM< W JXCBH <LJ L CKRH
4Q51
W---- HJ> MDBRT <L LBH RQ FPTJH N<WT W QWLH L> JCM< W JXCBH <LJ L CKRH
 
('Song_of_songs', 4, 4)
BHSA
K MGDL DWJD YW>RK BNWJ L TLPJWT >LP H MGN TLWJ <LJ-W KL CLVJ H GBWRJM
4Q106
K MGDL DWJD YW>RK BNWJ L TLPJWT >LP H MGN TLWJ ---BW KL CLVJ H GBWRJM
 
('Exodus', 30, 24)
BHSA
W QDH XMC M>WT B CQL H Q-DC W CMN ZJT HJN
2Q2
W QDH XMC M>WT B CQL H QWDC W CMN ZJT HJN
 
('Leviticus', 14, 33)
BHSA
W JDBR JHWH >L MCH W >L >HRN L >MR
4Q23
W JDBR JHWH >L MCH W >L >HRN L >MR
 
('Leviticus', 14, 33)
BHSA
W JDB

('Ecclesiastes', 5, 17)
BHSA
HNH >CR R>JTJ >NJ VWB >CR JPH L >KWL W L CTWT W L R>WT VWBH B K-L <MLW C J<ML TXT H CMC MSPR JMJ XJW >CR NTN LW H >LHJM KJ HW> XLQW
4Q109
HNH >CR R>JTJ >NJ VWB >CR JPH L >KWL W L CTWT W L R>WT VWBH B KWL <MLW C J<ML TXT H CMC MSPR JMJ XJW >CR NTN------------------ -L-W
 
('Exodus', 9, 23)
BHSA
W JV MCH >T MVHW <L H CMJM W JHWH NTN QLT W BRD W THLK >C >RYH W JMVR JHWH BRD <L >RY MYRJM
4Q14
W JV MCH >T MVHW <L H CMJM W JHWH NTN QLT W BRD W THLK >C >RYH W JMVR JHWH BRD <L >RY MYRJM
 
('Psalms', 77, 21)
BHSA
NXJT K Y>N <MK- B JD M-CH W >HR-N
11Q6
NXJT K Y>N <MKH B JD MWCH W >HRWN
 
('Job', 8, 17)
BHSA
<L GL CRCJW JSBKW BJT >BNJM JXZH
4Q100
<L GL CRCJW JSBKW---------------
 
('Isaiah', 6, 5)
BHSA
W >MR >WJ LJ KJ NDMJTJ KJ- >JC VM>- FPTJM >N-KJ W B TWK <M VM> FPTJM >N-KJ JWCB KJ- >T H MLK JHWH YB>WT R>W <JNJ
1Qisaa
W >MR >-J LJ KJ NDMJTJ KJ> >JC VM-H FPTJM >NWKJ W B TWK <M VM> FPTJM >NWKJ JWCB KJ> >T H MLK JHWH YB>WT R>W <JNJ
 
('Isaiah', 6, 5)
BHSA
W >MR >WJ LJ 

W >M L> JG>L <D ML>-T LW CNH TMJMH W QM H BJT >CR B <JR >CR L>- X--MH L YMJT-T L QNH >TW L DRTJW L> JY> B J-BL
11Q1
W >M L> JG>L <D ML>WT LW CNH TMJMH W QM H BJT >CR B <J----R L-W X WMH L YMJTWT L QNH >TW L DRTJW L> JY> B JWBL
 
('Deuteronomy', 6, 11)
BHSA
W BTJM ML>JM KL VWB >CR L> ML>T W BRT XYWBJM >CR L> XYBT KRMJM W ZJTJM >CR L> NV<T W >KLT W FB<T
4Q43
W BTJM ML>JM KL V-----------------------WB-- >CR L>------------- ----M-----L--------->---------T
 
('Jeremiah', 30, 18)
BHSA
KH >MR JHWH HNNJ CB CBWT >HLJ J<QWB W MCKN-TJW >RXM W NBNTH <JR <L TLH W >RMWN <L MCPVW JCB
4Q72
KH >MR JHWH HNNJ CB CBWT >HLJ J<QWB W MCKNWTJW >RXM W NBN-H <JR <L TLH W >RMWN <L MCPVW JCB
 
('Ecclesiastes', 5, 15)
BHSA
W GM ZH R<H XWLH K-L <MT C B> KN JLK W MH JTRWN LW C J<ML L RWX
4Q109
--GM ZH R<H XWLH KWL <MT C B> KN JLK W MH JTRWN LW C J<ML L RWX
 
('Psalms', 119, 44)
BHSA
W >CMRH TWRTK TMJD L <WLM W <D
1Q10
W >CMRH TWRTK TMJD L <WLM W <D
 
('Psalms', 119, 44)
BHSA
W >CMRH TWRT-K TMJD L <WLM W <D
4Q89
W >C

('Deuteronomy', 19, 15)
BHSA
L> JQWM <D >XD B >JC L KL <WN W L KL XV>T B KL XV> >CR JXV> <L PJ CNJ <DJM >W <L PJ CLCH <DJM JQWM DBR
4Q33
L> JQWM <D >XD B >JC L KL <WN W L KL XV>T B KL XV> >CR JXV----------------->--------------------------
 
('Deuteronomy', 19, 15)
BHSA
L> JQWM <D >XD B >JC L K-L <W-N W L K-L XV>T B KL XV> >CR JXV> <L PJ CNJ <DJM >W <L PJ CLCH <DJM JQWM DBR
4Q38a
L> JQWM <D >XD B >JC L KWL <WWN W L KWL XV>T B KL XV> >CR JXV> <L PJ CNJ <DJM >W <L PJ CLCH <DJM JQWM DBR
 
('Exodus', 14, 8)
BHSA
W JXZQ JHWH >T LB PR<H MLK MYRJM W JRDP >XRJ BNJ JFR>L W BNJ JFR>L JY>JM B JD RMH
4Q14
W JXZQ JHWH >T LB PR<H MLK MYRJM W JRDP >XRJ BNJ JFR>L W BNJ JFR>L JY>JM B JD RMH
 
('Exodus', 14, 8)
BHSA
W JXZQ JHWH >T LB PR<H MLK MYRJM W JRDP >XRJ BNJ JFR>L W BNJ JFR>L JY>JM B JD RMH
4Q22
W JXZQ JHWH >T LB PR<H MLK MYRJM W JRDP >XRJ BNJ JFR>L W BNJ JFR>L JY>JM B JD RMH
 
('Ezekiel', 1, 17)
BHSA
<L >RB<T RB<JHN B LKTM JLKW L> JSBW B LKTN
4Q74
<L >RB<T RB<JHN B LKTM JLKW L> JSBW B LKTN
 
('Ps

 
('Psalms', 145, 13)
BHSA
MLKWTK- MLKWT K-L <-LMJM W MMCLTK- B K-L DWR W DWR---------------------------------------------------------------------------------------------------------
11Q5
MLKWTKH MLKWT KWL <WLMJM W MMCLTKH B KWL DWR W DWR BRWK JHWH W BRWK CMW L <WLM W <D N>MN >LWHJM B DBRJW W XSJD B KWL M<FJW BRWK JHWH W BRWK CMW L <WLM W <D
 
('Genesis', 22, 15)
BHSA
W JQR> ML>K JHWH >L >BRHM CNJT MN H CMJM
1Q1
W JQR> ML>K JHWH >L >BRHM CNJT MN H CMJM
 
('Numbers', 11, 22)
BHSA
H Y>N W BQR JCXV LHM W MY> LHM >M >T KL DGJ H JM J>SP LHM W MY> LHM
4Q23
H Y>N W BQR JCXV------------------------------------- LHM W MY>----
 
('2_Samuel', 20, 13)
BHSA
K >CR HGH ----M----N H MSLH <BR K-L >JC- --->XRJ JW>B L RD-P >XRJ CB< BN BKRJ
4Q51
K >CR HGH >T <MF> MN H MSLH <BR KWL ---H <M >XRJ JW>B L RDWP >XRJ CB< BN BKRJ
 
('Zechariah', 2, 14)
BHSA
RNJ W FMXJ BT YJWN KJ HNNJ B> W CKNTJ B TWKK N>M JHWH
4Q80
RNJ W FMXJ BT Y---------------------J----W---N-------
 
('2_Samuel', 3, 27)
BHSA
W JCB >BNR XBRWN- 

## Postprocess data

Lexemes and parts of speech are collected and stored in lists, these are converted later to a pandas dataframe.

The distionary mapping_dict shows which lexemes in the dss package correspond with their new BHSA alternatives.

In [ ]:
tf_word_id = [] # tf index
scrolls = [] # scroll code
books = [] # biblical book name
chapters = [] # chapter number
verses = [] # verse number
words_dss = [] # consonantal representation of word on scroll
lexs_dss = [] # lexeme in dss package
lexs_etcbc = [] # corresponding etcbc lexeme
poss_dss = [] # POS in dss package
poss_etcbc = [] # corresponding etcbc POS

mapping_dict = collections.defaultdict(lambda: collections.defaultdict(list))

# loop over all words in the dss, and in each word over all consonants
for key in chars_per_word.keys():
    
    tf_word_id.append(ids_dss[key[1]][key[0]][key[2]])

    word = (''.join(chars_per_word[key])).replace("-", "")
    
    lexeme_dss, lexeme_etcbc = produce_value(key, chars_to_lexemes_etcbc, chars_to_lexemes_dss)
    pos_dss, pos_etcbc = produce_value(key, chars_to_pos_etcbc, chars_to_pos_dss)
    
    if lexeme_dss != '':
        mapping_dict[lexeme_dss][(key[0], key[1])].append(lexeme_etcbc)
    
    # collect info in lists
    scrolls.append(key[0])
    books.append(key[1][0])
    chapters.append(key[1][1])
    verses.append(key[1][2])
    words_dss.append(word)
    lexs_dss.append(lexeme_dss)
    lexs_etcbc.append(lexeme_etcbc)
    poss_dss.append(pos_dss)
    poss_etcbc.append(pos_etcbc)
    

In the following two cells, some empty cells are filled, using the second part of the procedure above.

In [ ]:
for index, lex in enumerate(lexs_etcbc):
  if lex == '':
    if lexs_dss[index] != '':
        
        all_candidates_lists = list((mapping_dict[lexs_dss[index]]).values())
        candidates_list = [item for sublist in all_candidates_lists for item in sublist]
        
        best_cand, count = most_frequent(candidates_list)

        lexs_etcbc[index] = best_cand

In [ ]:
mapping_lex_pos = collections.defaultdict(list)

for index, lex in enumerate(lexs_etcbc):
    
    if lex == "" or poss_etcbc[index] == "":
        continue
        
    else:
        mapping_lex_pos[lex].append(poss_etcbc[index])

In [ ]:
# to check: some lexemes have more than one pos, that needs to be corrected somehow

for key in mapping_lex_pos.keys():
    if len(set(mapping_lex_pos[key])) > 1:
        print(key, collections.Counter(mapping_lex_pos[key]))

Collect the data in a dataframe.

In [ ]:
df = pd.DataFrame(list(zip(tf_word_id, scrolls, books, chapters, verses, words_dss, lexs_dss, lexs_etcbc, poss_dss, poss_etcbc)), 
               columns =['tf_word_id', 'scroll','book','chapter', 'verse', 'g_cons', 'lex_dss', 'lex_etcbc', 'pos_dss', 'pos_etcbc']) 

df_new = df.sort_values(['book', 'scroll', 'chapter', 'verse'], ascending=[True, True, True, True])
df_new

Save the data in a csv file and print the mapping_dict. 

In [ ]:
df_new.to_csv("lexemes_pos_all_bib_books.csv", index=False)

In [ ]:
pprint(mapping_dict)